In [1]:
pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 981 kB 66.2 MB/s 
     |████████████████████████████████| 263 kB 64.4 MB/s 
     |████████████████████████████████| 3.8 MB 48.8 MB/s 
     |████████████████████████████████| 1.3 MB 59.9 MB/s 
     |████████████████████████████████| 468 kB 64.5 MB/s 
     |████████████████████████████████| 6.6 MB 51.8 MB/s 
     |████████████████████████████████| 120 kB 44.3 MB/s 
     |████████████████████████████████| 880 kB 69.3 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.7-py3-none-any.whl size=25312842 sha256=b8fd3e71516712905f9e105f1d8e3700e49c08df327db42d4d7b4f4fe82016b4
  Stored in directory: /root/.cache/pip/wheels/e5/98/8e/ce355dcb92451e85fab93f7ea2da068843e93e703928cd06fb
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=1768b7cb29099a0a5f7fbda7

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.8.2'

In [4]:
from google.colab import files
upload = files.upload()

Saving test.xlsx to test.xlsx
Saving train.xlsx to train.xlsx


In [5]:
import numpy as np
import pandas as pd
import ktrain
from ktrain import text

In [6]:
df_test = pd.read_excel('/content/MovieTest.xlsx', dtype= str)
df_train = pd.read_excel('/content/MovieTrain.xlsx', dtype = str)

In [7]:
df_test.shape, df_train.shape

((25000, 2), (25000, 2))

In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [9]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = df_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [10]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400


Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

done.


In [11]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [12]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - 1597s 380ms/step - loss: 0.2881 - accuracy: 0.8802 - val_loss: 0.1530 - val_accuracy: 0.9450
Epoch 2/2
4167/4167 [==============================] - 1593s 381ms/step - loss: 0.1590 - accuracy: 0.9424 - val_loss: 0.0656 - val_accuracy: 0.9826


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [14]:
data = ['this movie was okay, the story was fine.',
        'the movie really sucked. there is not plot and acting was bad']

In [15]:
predictor.predict(data)

['pos', 'neg']

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [19]:
predictor.save('/content/drive/MyDrive/distilbert')

In [20]:
predictor.get_classes()

['neg', 'pos']

In [21]:
predictor.predict(data, return_proba=True)

array([[0.30972368, 0.6902763 ],
       [0.99620074, 0.00379924]], dtype=float32)

In [22]:
ktrain.__version__

'0.31.7'